# 5章 誤差逆伝播法

- 数値微分はシンプルだけど，計算に時間がかかる
- それを効率的に行う「誤差逆伝播法」を紹介
- 「数式」ではなく「計算グラフ」で視覚的に誤差逆伝播法を理解する
    - andrej Karpathy のブログ
    - スタンフォード大学のディープラーニングの授業「CS231n」

## 5.1 計算グラフ

- 計算の過程をグラフによって表したもの
- ノードとエッジで構成される
    - ノード：点
    - エッジ：線

### 5.1.1 計算グラフで解く

- 問1. 太郎くんはスーパーで1個100円のリンゴを2個買いました．支払う金額を求めなさい．ただし，消費税が10%適用されるものとします．

計算グラフでは，掛け算をこのように書くことができる．

<img src="images/5_1.JPG">

また「リンゴの個数」と「消費税」を変数としておくと，こんな風になる．

<img src="images/5_2.JPG">

- 問2. 太郎くんはスーパーでリンゴを2個，みかんを3個買いました．リンゴは1個100円，みかんは1個150円です．消費税が10%かかるものとして，支払う金額を求めなさい．

この問題を解く計算グラフ書いてみましょう．

答え<hidden img src="images/5_3.JPG">

ここまでの流れを見ると，計算グラフを使って問題を解くには
- 計算グラフを構築する
- 計算グラフ上で計算を左から右へ進める

この「計算を左から右へ進める」ことを「順伝搬」

この逆方向への伝搬も考えることができて，それを「逆伝搬」

「逆伝搬」はこの先の微分の計算に重要な働きをする

### 5.1.2 局所的な計算

- 計算グラフの特徴：「局所的な計算」を伝搬することによって最終的な結果を得ることができる
    - 4,200 という数字の計算には 4,000 や 200 がどうやって計算されてきたかを考える必要がない

ひとつひとつは単純な計算でも，最終的に複雑な計算の結果を得ることができる

<img src="images/5_4.JPG">

### 5.1.3 なぜ計算グラフで解くのか？

- 問題を単純化できる
- 途中の計算の結果を保持することができる

でもここで一番大事なのは
- 「微分」を効率よく計算できること

問1をもう一度使って

「リンゴの値段が値上がりした場合，最終的な支払金額にどのように影響するか」->「リンゴの値段に関する支払金額の微分」

を考えてみると，計算グラフでの微分の結果は線の下の値のようになる．

<img src="images/5_5.JPG">

このように途中の計算の結果（ここでは 1.1）を利用して，より手前の微分の値（ここでは 2.2）を求めることができる．

## 5.2 連鎖律

「逆伝搬」の計算は「局所的な微分」を右から左へ伝達することで行われる．

この「局所的な微分」を伝達する原理は「連鎖律」によるもの．

### 5.2.1 計算グラフの逆伝搬

- $y = f\left(x\right)$ の逆伝搬

<img src="images/5_6.JPG">

### 5.2.2 連鎖律とは

#### 連鎖律（Chain rule）の説明には，合成関数が必要

- 合成関数：複数の関数によって構成される関数

例えばこんな関数は

$$z = (x + y)^2$$

こんな風に2つの式に分けられる．

$$z = t^2$$
$$t = x+ y$$

連鎖律とは合成関数の微分についての性質（連鎖率の原理）：
- ある関数が合成関数で表される場合，その合成関数の微分は，合成関数を構成するそれぞれの関数の微分の積によって表すことができる．

式で書くと次のように表される．

$$\frac{\partial z}{\partial x} = \frac{\partial z}{\partial t} \frac{\partial t}{\partial x}$$

#### 連鎖律を使って $\frac{\partial z}{\partial x}$ を計算してみる

分けた式の微分をそれぞれ求める．

$$\frac{\partial z}{\partial t} = \frac{\partial}{\partial t} t^2 = 2 t$$

$$\frac{\partial t}{\partial x} = \frac{\partial}{\partial x} \left( x + y \right) = 1$$

これらの結果を使うと，簡単に $\frac{\partial z}{\partial x}$ が計算できる．

$$\frac{\partial z}{\partial x} = \frac{\partial z}{\partial t} \frac{\partial t}{\partial x} = 2t \cdot 1 = 2 \left( x + y \right)$$

### 5.2.3 連鎖律と計算グラフ

先程行った連鎖率を使った計算を，計算グラフを用いて表すと図5-7のようになる．

<img src="images/5_7.JPG">

具体的な計算結果を当てはめると図5-8のようになる．

<img src="images/5_8.JPG">

## 5.3 逆伝搬

「+」や「×」などの演算を例に，逆伝搬の仕組みについて見ていく．

### 5.3.1 加算ノードの逆伝搬

#### $z = x + y$ の逆伝搬をみていく

まずは微分．

$$ \frac{\partial z}{\partial x} = 1$$

$$ \frac{\partial z}{\partial y} = 1$$

この結果を使って，上流から伝わった値 $L$ の $z$ に関する微分を計算すると図5-9のようになる．

計算グラフの下流には，どちらもそのまま $\frac{\partial L}{\partial z}$ が流れる．

<img src="images/5_9.JPG">

実際にはこんな風に「何らかの計算」の途中に存在することが多い．「何らかの計算」がどんなに複雑であっても，先程と同じように $\frac{\partial L}{\partial z}$ を下流に流すだけでよい．

<img src="images/5_10.JPG">

数値例 $x = 10, y = 5$ では，図5-11のようになる．※$\frac{\partial L}{\partial z} = 1.3$

<img src="images/5_11.JPG">

### 5.3.2 乗算ノードの逆伝搬

#### $z = xy$ の逆伝搬をみていく

まずは微分．

$$ \frac{\partial z}{\partial x} = y$$

$$ \frac{\partial z}{\partial y} = x$$

この結果を使って，上流から伝わった値 $L$ の $z$ に関する微分を計算すると図5-12のようになる．

<img src="images/5_12.JPG">

数値例 $x = 10, y = 5$ では，図5-13のようになる．※$\frac{\partial L}{\partial z} = 1.3$

<img src="images/5_13.JPG">

### 5.3.3 リンゴの例

#### 3つの変数（リンゴの値段，個数，消費税）それぞれが最終的な支払金額にどのように影響するか

- リンゴの値段：リンゴの値段に関する支払金額の微分
- リンゴの個数：リンゴの個数に関する支払金額の微分
- 消費税：消費税に関する支払金額の微分

計算グラフでの計算結果は図5-14のようになる．

消費税に関する微分の値が特に大きく，支払金額への影響が他よりも大きいことを示している．

<img src="images/5_14.JPG">

#### 練習問題

<img src="images/5_15.JPG">